In [1]:
using JSON, ProgressMeter, JLD, LightGraphs 

## Community Detection on twitter network using non-negative matrix factorization and graph regularization.
This implements the `DualNMF` algorithm in this paper: [Community Detection in Political Twitter Networks using Nonnegative Matrix Factorization Methods](https://arxiv.org/abs/1608.01771)

Prerequisites:
- the retweet graph as a LightGraphs object
- the user/graph matrix

In [18]:
user_word = JLD.load("/media/henripal/hd1/data/new_mat.jld", "new_mat");
graph = JLD.load("/media/henripal/hd1/data/graph.jld", "graph");

In [19]:
size(user_word,2)

5132

### Building a word/word similarity matrix from the user/word matrix
We iterate on columns because its faster and memory friendly in Julia:

In [20]:
function sparse_similarity(m::SparseMatrixCSC)::SparseMatrixCSC
    normalized_user_word = spzeros(size(m)...)
    norms = [norm(m[:,i]) for i in 1:size(m,2)]
    @showprogress for (col,s) in enumerate(norms)
        s == 0 && continue # What does a "normalized" column with a sum of zero look like?
        normalized_user_word[:,col] = m[:,col]/s
    end
    return normalized_user_word' * normalized_user_word
end


# this builds a LightGraphs graph from a similartiy matrix
function build_graph_from_similarity(similarity::Matrix, cutoff::Float64)::Graph
    length = size(similarity, 1)
    graph = Graph(length)
    for i in 1:length
        for j in 1:i-1
            similarity[i, j] > cutoff && add_edge!(graph, i, j)
        end
    end
    graph
end
    

sparse_similarity (generic function with 1 method)

In [21]:
# this succesively builds the similarity matrix then
# builds the graph from the similarity matrix

cutoff = .4
similarity = sparse_similarity(user_word)
word_graph = build_graph_from_similarity(full(similarity), cutoff)

Progress: 100%|█████████████████████████████████████████| Time: 0:04:16


5132×5132 sparse matrix with 24981772 Float64 nonzero entries:
	[1   ,    1]  =  1.0
	[2   ,    1]  =  0.104883
	[3   ,    1]  =  0.0077162
	[4   ,    1]  =  0.134559
	[5   ,    1]  =  0.0689679
	[6   ,    1]  =  0.0223928
	[7   ,    1]  =  0.00913363
	[8   ,    1]  =  0.146813
	[9   ,    1]  =  0.0788164
	[10  ,    1]  =  0.116557
	⋮
	[5122, 5132]  =  0.120978
	[5123, 5132]  =  0.177761
	[5124, 5132]  =  0.0172602
	[5125, 5132]  =  0.266282
	[5126, 5132]  =  0.195395
	[5127, 5132]  =  0.0338713
	[5128, 5132]  =  0.138883
	[5129, 5132]  =  0.127958
	[5130, 5132]  =  0.300903
	[5131, 5132]  =  0.229402
	[5132, 5132]  =  1.0

### Algorithm
- calculate the Laplacian matrices
- define the update functions
- define the cost function
- iterate the update functions

Some remarks: the cost function is quite expensive, so we do not calculate it everytime. It would be maybe useful to adaptively calculate it?

Some of the helper functions are a little strange looking; this is because these matrices are huge, some are not sparse, and the memory usage can get a little out of control. Hence the column iterations, some precalculations, etc..

In [22]:
# calculating the laplacian matrices and plusminus laplacian:
L_c = laplacian_matrix(graph)
word_laplacian = laplacian_matrix(word_graph)
wl_plus = (abs(word_laplacian)+word_laplacian)/2
wl_minus = (abs(word_laplacian)-word_laplacian)/2
gl_plus = (abs(L_c)+L_c)/2
gl_minus = (abs(L_c)-L_c)/2

1205559×1205559 sparse matrix with 7484416 Int64 nonzero entries:
	[1      ,       1]  =  8
	[2      ,       1]  =  -1
	[3      ,       1]  =  -1
	[4      ,       1]  =  -1
	[5      ,       1]  =  -1
	[6      ,       1]  =  -1
	[7      ,       1]  =  -1
	[8      ,       1]  =  -1
	[9      ,       1]  =  -1
	[1      ,       2]  =  -1
	⋮
	[1205555, 1205555]  =  2
	[234    , 1205556]  =  -1
	[235    , 1205556]  =  -1
	[1205556, 1205556]  =  2
	[11575  , 1205557]  =  -1
	[1205557, 1205557]  =  1
	[94     , 1205558]  =  -1
	[1205558, 1205558]  =  2
	[1205559, 1205558]  =  -1
	[1205558, 1205559]  =  -1
	[1205559, 1205559]  =  1

In [28]:
# parameters and initializing W and U
clusters = 60
α = 10
β = 10
users = size(L_c,1)
words = size(word_laplacian ,1)
W = .5 * spones(sprand(words, clusters, .5))
U = .5 * spones(sprand(users, clusters, .5))


1205559×60 sparse matrix with 36172295 Float64 nonzero entries:
	[2      ,       1]  =  0.5
	[4      ,       1]  =  0.5
	[6      ,       1]  =  0.5
	[9      ,       1]  =  0.5
	[10     ,       1]  =  0.5
	[13     ,       1]  =  0.5
	[14     ,       1]  =  0.5
	[15     ,       1]  =  0.5
	[16     ,       1]  =  0.5
	[19     ,       1]  =  0.5
	⋮
	[1205542,      60]  =  0.5
	[1205547,      60]  =  0.5
	[1205548,      60]  =  0.5
	[1205549,      60]  =  0.5
	[1205550,      60]  =  0.5
	[1205551,      60]  =  0.5
	[1205552,      60]  =  0.5
	[1205553,      60]  =  0.5
	[1205554,      60]  =  0.5
	[1205558,      60]  =  0.5
	[1205559,      60]  =  0.5

In [30]:
# memory friendly update functions

function update_U(U::SparseMatrixCSC, W::SparseMatrixCSC)::SparseMatrixCSC
    WpW = W' * W
    return U .* sqrt((user_word * W + α * gl_minus * U) ./ (U * WpW + α * gl_plus * U))
end

function update_W(U::SparseMatrixCSC, W::SparseMatrixCSC)::SparseMatrixCSC
    UpU = U' * U
    return W .* sqrt((user_word' * U + β * wl_minus * W) ./ (W * UpU + β * wl_plus * W))
end

update_U (generic function with 1 method)

In [32]:
# memory friendly frobenius norms and objective functions

function my_frobenius(uw::SparseMatrixCSC, U::SparseMatrixCSC, W::SparseMatrixCSC)::Float64
    (users, words) = size(uw)
    wp = W'
    clusters = size(U,2)
    result = 0
    @showprogress for j in 1:words
        uwp_j = U*wp[:, j]
        result += norm(uw[:, j] - uwp_j)^2
    end
    result
end

function obj(U::SparseMatrixCSC, W::SparseMatrixCSC)::Float64
    my_frobenius(user_word, U, W) + α * trace(U' * L_c * U) + β * trace(W' * word_laplacian * W)
end

obj (generic function with 1 method)

This is where we run the algorithm. Somewhat time intensive but not crazily so

In [47]:
tolerance = .05
delta = 1000
stride = 10

err = obj(U, W)

while delta > tolerance
    for i in 1:stride
        U = update_U(U, W);
        W = update_W(U, W);
    end
    newerr = obj(U,W)
    delta = abs(newerr - err)
    err = newerr
end

Progress: 100%|█████████████████████████████████████████| Time: 0:09:33


In [49]:
JLD.save("/media/henripal/hd1/data/U_60.jld", "U_60", U)

In [51]:
# another helper functions, assigns the communities based on the highest probability of being in that community

function assign_communities(u::SparseMatrixCSC)
    (n_user, n_cluster) = size(u)
    communities = Array{Int64,1}(n_user)
    @showprogress for user in 1:n_user
        communities[user] = indmax(u[user, :])
    end
    communities
end


assign_communities (generic function with 1 method)

In [52]:
comm = assign_communities(U)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


1205559-element Array{Int64,1}:
 19
 27
 31
  7
 19
 19
 59
 22
 18
 31
 31
 25
 44
  ⋮
 41
 25
 41
 21
 51
 57
 10
 17
 31
  9
  2
 30

In [53]:
using Plots

In [55]:
histogram(comm, nbins = 60)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 0 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## Some post-processing to vizualize data using projector, and restrict ourselves to the 10k largest accounts.
This is totally in rough draft form

In [10]:
U_60 = JLD.load("/media/henripal/hd1/data/U_60.jld", "U_60")

1205559×60 sparse matrix with 35334538 Float64 nonzero entries:
	[2      ,       1]  =  0.0495808
	[6      ,       1]  =  0.00918191
	[9      ,       1]  =  0.123767
	[10     ,       1]  =  0.000630212
	[13     ,       1]  =  0.00385952
	[14     ,       1]  =  0.00347653
	[15     ,       1]  =  0.00244331
	[16     ,       1]  =  0.000328876
	[19     ,       1]  =  0.00363582
	[20     ,       1]  =  0.00131454
	⋮
	[1205542,      60]  =  8.94298e-39
	[1205547,      60]  =  5.19441e-202
	[1205548,      60]  =  2.42452e-200
	[1205549,      60]  =  7.16569e-5
	[1205550,      60]  =  7.76278e-30
	[1205551,      60]  =  0.0097963
	[1205552,      60]  =  5.43669e-8
	[1205553,      60]  =  1.072e-5
	[1205554,      60]  =  6.15074e-75
	[1205558,      60]  =  0.000951541
	[1205559,      60]  =  0.000741547

In [12]:
using DataFrames

In [25]:
name_followers = readtable("/media/henripal/hd1/data/name_to_follower.csv", header = false);

In [26]:
rename!(name_followers,:x1,:name)
rename!(name_followers,:x2, :followers)

,name,followers
1,GavaironJ,5
2,bocchijoto,1834
3,cannabinolsen,1
4,angelman61,32
5,alex_latrice21,199
6,turnipkween,242
7,EveMorante,747
8,mwutley,113
9,LetsCllnk,59
10,positivelytaco,173


In [32]:
sort!(name_followers, cols= :followers, rev = true);

In [35]:
name_followers = name_followers[1:10000,:]

,name,followers
1,MileyCyrus,31598990
2,TheEconomist,18303980
3,POTUS,14277895
4,funnyordie,13936034
5,TIME,12715214
6,ArvindKejriwal,10276363
7,SarahKSilverman,9776677
8,jk_rowling,9056612
9,HuffingtonPost,8868956
10,people,7546839


In [36]:
name_followers[:ind] = [name_to_index[n] for n in name_followers[:name]]

10000-element Array{Int64,1}:
 1105228
  400654
  430428
  519328
   18442
 1024618
 1058073
  701067
  521599
  441624
  928554
   21126
  883751
       ⋮
 1056759
  389300
  861936
 1060834
  628955
  783077
  146505
  687415
  538132
  373521
  903695
  727673

In [38]:
user_vectors = Array{Float64,2}(10000, 60)

10000×60 Array{Float64,2}:
 5.31633e-318  5.26693e-318  5.21753e-318  …  4.09076e-319  3.47501e-319
 5.31633e-318  5.26692e-318  5.21753e-318     4.09081e-319  3.47506e-319
 5.31632e-318  5.26692e-318  5.21751e-318     4.09086e-319  3.47511e-319
 5.31632e-318  5.26691e-318  5.21751e-318     4.09091e-319  3.47516e-319
 5.31631e-318  5.2669e-318   5.21748e-318     4.09096e-319  3.47521e-319
 5.31631e-318  5.26691e-318  5.21749e-318  …  4.09101e-319  3.47526e-319
 5.3163e-318   5.26689e-318  5.21749e-318     4.09106e-319  3.47531e-319
 5.3163e-318   5.2669e-318   5.2175e-318      4.09111e-319  3.47536e-319
 5.31629e-318  5.26689e-318  5.2175e-318      4.09116e-319  3.47541e-319
 5.31629e-318  5.26688e-318  5.21747e-318     4.09121e-319  3.47546e-319
 5.31628e-318  5.26688e-318  5.21747e-318  …  4.09126e-319  3.4755e-319 
 5.31628e-318  5.26687e-318  5.21748e-318     4.09131e-319  3.47555e-319
 5.31627e-318  5.26687e-318  5.21745e-318     4.09136e-319  3.4756e-319 
 ⋮                      

In [41]:
for i in 1:10000, j in 1:60
    user_vectors[i, j] = U_60[name_followers[:ind][i], j]
end

In [43]:
user_cluster = DataFrame(user_vectors)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60
1,0.0003308432856116264,0.00023556579292053693,2.0446540121546803e-10,0.0,0.0,0.0,0.0,0.0023846671602313157,0.0,0.0,1.6802939987742255e-6,0.0,2.014624184008981e-5,1.3887591951182302e-8,3.064388919408614e-42,0.0,0.00012192591171959283,0.0028448940191847525,8.754336407056734e-5,0.0,0.044498164798964075,0.00014910775143976663,0.0,0.0062730740032405155,0.0,4.573211742164153e-6,0.003984731289917218,0.0,0.0,0.0,0.0,0.0010430127076793336,0.0,0.0,6.587998420877805e-5,0.0,0.0,0.0,0.005361382893127455,0.0,0.0007355158623978776,0.0008112884222575503,0.0,5.8160241596373914e-5,0.0,0.0,0.0003344268464441192,0.00553142754076147,0.0006128464480507761,0.0014175367508819355,0.0,0.0,0.0,0.0,0.0008470209740478379,0.004493430134649289,0.0023314149353175326,0.0,0.0,0.0
2,0.012185938992453698,0.007520346598927166,0.0,0.0,0.0,0.0,0.002790785545341013,0.000439382143195168,0.0,0.0,0.0,0.0,0.0,4.705423402757473e-39,1.2545409526519993e-7,0.010573547372347187,0.0,0.0,8.530843319542127e-27,0.0,0.0015000041049208408,0.0,0.0,0.0,0.0,0.0,0.00046473389681072724,0.0,0.0,1.1834661896183532e-33,0.0,0.00911536167853829,1.063352318605058e-44,0.002258826631600358,0.0,0.00039758467685113924,0.0020938145798602256,1.2654353806380195e-147,0.0,0.0011614075145673446,0.0,1.5617438169482512e-77,4.2474198281799115e-5,0.0,3.425544054559048e-5,0.0,6.867498516563607e-39,0.0026496856867044482,0.0,0.0,6.157052518337497e-17,0.0,4.154087665370867e-30,0.0,8.69547615838423e-24,0.0,8.984054254422174e-28,4.607879997896263e-5,0.0,3.905345380474349e-20
3,0.0,0.0,4.020239757033454e-24,0.0,0.0,7.1523067998953e-5,0.0,0.0005110360732394288,0.0,0.0,3.024365460555043e-6,0.0,3.1404478241281815e-5,0.0,0.0,0.0003074021603834905,0.0,0.0,0.0,0.0,6.289792525958274e-7,0.003557902637807497,0.0,0.0,1.2213750773923446e-14,0.025782587377123693,0.0,5.770156877870895e-6,6.53461782663682e-7,2.2871680716142906e-23,0.0,0.000104118694924968,2.236325870673125e-25,3.2812142603073855e-7,0.0,0.003517295057961414,0.0,1.3710349762929702e-142,1.7124600972339375e-30,0.0010301990333658104,0.0,3.1587700870857397e-16,0.006979161111813753,0.010136481932131538,0.0,0.0,0.0,0.0,0.002934064627641937,0.0,0.0,0.0003102983439209935,0.0,0.0,0.0,0.0,1.319297570782638e-5,0.0,0.0,0.0023441462028319865
4,0.0017747488671678486,0.0,3.8932615628673544e-26,0.0,0.0,0.0,0.0020622585463183594,0.0,0.0,0.0,0.0,6.886827065372384e-26,0.0,0.016605417395298003,0.0,0.005625564296704098,0.0007861195454822195,0.0,8.296795781269439e-19,0.0,0.004489961625987464,0.0,8.987477612678408e-5,0.0,4.544902190476385e-16,0.0,0.0,0.0017504705214968948,0.00022072143368432497,0.0,0.0,0.0,5.312240722715251e-31,0.0,0.0,0.0,0.0008721710688863963,0.0,0.0,0.0,0.042418985581581616,0.0,0.00029934319668378074,0.0,0.00012346971071342522,4.4390717534201144e-58,4.632406866563366e-29,0.0,0.0,0.00012421581861712244,1.984309035718981e-17,0.0,4.538956728803023e-22,0.0,0.0,0.0,2.904558844407672e-21,0.00024209566710284213,0.0037723766604976405,0.0
5,0.009788426324280628,0.0,2.3076090435205563e-9,0.0,0.0,0.0003145196539987476,0.012325762995579496,0.0,0.015500435540805345,0.0,2.672768750511227e-5,0.009603790613093237,0.0,0.0,0.0,0.01320465891791403,0.01038910379979191,0.017645209904107263,0.0,0.0008529141569550366,0.0,0.0066143481924247245,0.0,7.810202183096931e-13,0.05294513664114524,0.00011567592144047174,0.0,0.0,0.025747690785669696,0.0,0.06523723681043811,0.0,0.0,0.0,0.0,0.0,0.010207420264402169,3.0045663718339554e-5,0.0,0.0,0.022034786479435728,0.0,0.0,0.0,0.0039776570924686845,0.0017355855189742848,0.0,0.0,0.007821576465136837,0.0,0.0,2.1126924436052718e-13,0.0002901655326990168,0.0,0.0,0.0,0.0,0.0,0.0,0.0004784013468876284
6,0.0,0.0,2.182879405819539e-58,0.004946402200529859,0.0,0.0018532675317762446,0.0,0.0,9.249687767222297e-10,0.0153230134706511

In [62]:
class = [indmax(user_vectors[i,:]) for i in 1:10000];
class_freq = zeros(60)
for class_no in class
    class_freq[class_no] = class_freq[class_no]+1
end

In [74]:
best_classes = [i for i in 1:60 if class_freq[i] > 250]

10-element Array{Int64,1}:
  7
 10
 17
 21
 25
 29
 31
 34
 41
 44

In [78]:
new_class = [indmax(user_vectors[i, best_classes]) for i in 1:10000]

10000-element Array{Int64,1}:
  4
  1
 10
  9
  7
  3
  1
  9
  7
  7
  3
  7
  3
  ⋮
  7
  7
  5
  7
  9
  7
  5
  5
  8
  2
  6
  2

In [79]:
name_followers[:class] = string.(new_class)

10000-element Array{String,1}:
 "4" 
 "1" 
 "10"
 "9" 
 "7" 
 "3" 
 "1" 
 "9" 
 "7" 
 "7" 
 "3" 
 "7" 
 "3" 
 ⋮   
 "7" 
 "7" 
 "5" 
 "7" 
 "9" 
 "7" 
 "5" 
 "5" 
 "8" 
 "2" 
 "6" 
 "2" 

In [80]:

writetable("/media/henripal/hd1/data/attributes.tsv",name_followers[:,[:name, :class, :followers]])

In [53]:

writetable("/media/henripal/hd1/data/vectors.tsv",user_cluster, header = false)

In [20]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,name
1,0.0,0.0,0.0,0.0,0.0051725596147227,0.026945830598873732,0.10775474686882859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0003681329605147508,0.05019964953095866,0.0,0.01634360690677292,0.2769832659687579,0.0,0.0,0.0,0.011138582784611212,0.0,0.0,0.0,0.0,0.050674768950433255,0.0,0.0,0.0,0.011685303315783595,5.982337113625717e-7,0.025341281084838997,0.0,0.1397876310794631,0.0,2.557728206998708e-50,0.022798613032828093,0.11137848732635668,0.033592642112617435,1.1281923594343566e-20,0.0,0.0,0.0,0.00031783142602080953,0.0,0.0,0.0,0.08351004447658451,0.009862793276377544,0.0015407854886804928,0.014428228467605742,0.0,0.0,0.01562588099842397,0.0,0.03947191471247384,0.09654327952085223,0.025225820001461275,Deborah87958167
2,0.049580753988165444,0.2508836444918074,0.0,0.0,0.0,0.0,0.04489051981720582,0.1469489793379341,0.006060481798182574,0.029514101635304844,0.0890455518754558,0.0,0.0,0.0,0.0002022396002893607,0.005661225864393524,0.0,0.0,0.0,0.0,0.0,0.058310273766597205,0.0,0.0,0.0,0.0,0.3711326910656078,0.0,0.18335415663300975,0.0,0.0,0.0,0.0033604598334223044,0.0,0.0,0.0,0.0,0.0,0.04542499610223528,0.07414130821506772,0.08966204938609933,0.0,0.11555116359807116,0.0,0.17541942420018627,0.0,0.0,0.0,0.0,0.2132898499804196,0.0,0.0031261188507839795,0.0,0.07288551281835083,0.0,0.08977849524134132,0.0,0.05134670979062862,0.08354029060885172,0.0,texasfarmgirl1836
3,0.0,8.713427609010385e-9,0.0,0.0,2.506603772409537e-16,0.0014415837633974843,0.009184575106136393,0.0014634597958376704,0.0,0.0,1.4087849542070368e-9,3.0556874831156914e-6,0.0,2.081966235264321e-18,0.0,0.020324572789786673,0.0,0.0,0.0,0.0,0.0,0.0,0.016042098750867012,0.0,0.0,5.010620238826393e-7,0.0,0.006732738181303161,0.0,0.004281777076742496,0.0483308829964388,0.0,1.7176459235562508e-11,0.0,0.0,0.0,0.007889798225674597,3.1674231391041387e-9,0.028718489356872427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0021219079093766796,4.806771499892348e-9,0.0,0.0034717900607107335,3.0484456781940202e-5,2.9008845144491245e-12,0.0,0.0,0.0,0.0,0.0017901387836809907,0.0,0.0,0.0,Squatch
4,0.0,0.0,0.0,0.0,6.757009503150856e-9,0.0,0.004290304298363777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.349674366843527e-8,0.0,1.8740760403039192e-19,0.0,0.0,0.0,0.0,2.647414737804112e-16,0.0,0.0,0.0,0.0,7.659799678820569e-12,0.0,0.0,0.0,6.995064178052164e-13,0.0,4.167343897568493e-10,0.0,0.0,0.0,0.0,0.0,0.0,3.690302282820236e-9,6.2441708347947e-253,0.0,0.0,0.0,3.9245758136038415e-24,0.0,0.0,0.0,4.468493945327262e-6,0.0,3.943752366065573e-7,0.0,0.0,0.0,0.0,0.0,3.399631642317278e-9,0.0,0.00033363499990994886,Lu Who
5,0.0,0.0,2.978697870558589e-20,0.05610419050580491,0.0,0.0,0.1006611078941685,0.0,0.0,0.0,0.0017628746836431215,1.844669806182308e-20,0.002818507875767357,0.0,0.0,0.0,0.0291893271717699,0.0,0.2695722422915973,0.0,0.029762575540874536,0.07906190763660918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046634138804370874,0.0,0.0,0.0,0.0,0.0,0.16956119494466845,0.0,4.626771949675073e-103,0.0,0.0,0.0,8.911343599386976e-17,0.0,0.0,0.05665828725416513,0.0004112506664022114,0.0,0.02254564482163113,0.0,0.0542322502647718,0.004808077603267288,0.0011385726686938334,0.0,0.014768789225195375,0.0,0.0,0.0008794133262373876,0.044051796341177264,0.0862211039196512,0.0,SongsOfLaredo
6,0.009181913913791745,0.0,1.1601476735579242e-5,0.0,0.00035113379363617604,0.010457945596979469,0.01890423467102866,0.024182000824113852,0.0081186890821123,0.019336573934026782,0.0,0.0,0.0020369611070950775,0.0002501484514695846,0.0,0.0,0.007764641419741867,0.0,0.07971543948692807,0.014415242658270294,0.03100934410246285,0.0,0.025303611760274857,0.0,0.04141569848179266,0.01773688481428575,0.028663842332646797,0.0,0.023666765466495245,0.0,0.045465174523809285,0.017773289768186835,0.0003725663814940007,0.02036893439985751,0.0,0.0,0.01496255824068227,0.000